In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


In [2]:
## Load dataset with data_loader
data_dir    = "data/"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt"

loading preprocessed files


In [3]:
## vocabs and chars
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader.vocab' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )

type of 'data_loader.vocab' is <class 'dict'>, length is 75
type of 'data_loader.chars' is <class 'tuple'>, length is 75


In [4]:
## Vocab dictionary (Char -> Index)
print (data_loader.vocab)

{'ᴥ': 0, ' ': 1, 'ㅇ': 2, 'ㅏ': 3, 'ㄴ': 4, 'ㄹ': 5, 'ㅣ': 6, 'ㄱ': 7, 'ㅡ': 8, 'ㅗ': 9, 'ㅅ': 10, 'ㅓ': 11, 'ㅁ': 12, 'ㄷ': 13, 'ㅎ': 14, 'ㅈ': 15, 'ㅜ': 16, 'ㅂ': 17, 'ㅕ': 18, '\n': 19, 'ㅔ': 20, 'ㅐ': 21, 'ㅆ': 22, 'ㅊ': 23, '.': 24, 'ㅢ': 25, 'ㅘ': 26, ',': 27, '"': 28, 'ㄲ': 29, 'ㅌ': 30, 'ㅍ': 31, 'ㄸ': 32, 'ㅚ': 33, 'ㅑ': 34, 'ㅟ': 35, 'ㅝ': 36, 'ㅠ': 37, 'ㅛ': 38, 'ㅖ': 39, 'ㅉ': 40, '?': 41, 'ㅋ': 42, 'ㅄ': 43, 'ㄶ': 44, '(': 45, ')': 46, 'ㅃ': 47, 'ㄺ': 48, "'": 49, 'ㄵ': 50, 'ㅀ': 51, '!': 52, 'ㅙ': 53, 'ㄼ': 54, 'ㄻ': 55, 'ㄿ': 56, ':': 57, '1': 58, '6': 59, '2': 60, 'ㄳ': 61, 'ㅞ': 62, '9': 63, '5': 64, '-': 65, '4': 66, '3': 67, '8': 68, '7': 69, '_': 70, 'ㄾ': 71, 'ㅒ': 72, '0': 73, '>': 74}


In [5]:
## Char index (Index -> Char)
print (data_loader.chars)
# USAGE
print (data_loader.chars[0])

('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 'ㅆ', 'ㅊ', '.', 'ㅢ', 'ㅘ', ',', '"', 'ㄲ', 'ㅌ', 'ㅍ', 'ㄸ', 'ㅚ', 'ㅑ', 'ㅟ', 'ㅝ', 'ㅠ', 'ㅛ', 'ㅖ', 'ㅉ', '?', 'ㅋ', 'ㅄ', 'ㄶ', '(', ')', 'ㅃ', 'ㄺ', "'", 'ㄵ', 'ㅀ', '!', 'ㅙ', 'ㄼ', 'ㄻ', 'ㄿ', ':', '1', '6', '2', 'ㄳ', 'ㅞ', '9', '5', '-', '4', '3', '8', '7', '_', 'ㄾ', 'ㅒ', '0', '>')
ᴥ


In [6]:
## Training batch (IMPORTANT!!!)
x, y = data_loader.next_batch()
print ("Type of 'x' is %s. Shape is %s" % (type(x), x.shape,))
print ("x looks like \n%s" % (x))
print
print ("Type of 'y' is %s. Shape is %s" % (type(y), y.shape,))
print ("y looks like \n%s" % (y))

Type of 'x' is <class 'numpy.ndarray'>. Shape is (50, 50)
x looks like 
[[19 19  1 ... 23  0  4]
 [ 1 19 42 ...  6  0  2]
 [ 2  8  0 ... 19  1  1]
 ...
 [ 1  7  8 ...  2  6  0]
 [ 6  4  0 ... 17  9  7]
 [ 1 19 17 ... 17 16  5]]
Type of 'y' is <class 'numpy.ndarray'>. Shape is (50, 50)
y looks like 
[[19  1  1 ...  0  4  3]
 [19 42 11 ...  0  2 20]
 [ 8  0  5 ...  1  1 10]
 ...
 [ 7  8  0 ...  6  0  1]
 [ 4  0 12 ...  9  7  0]
 [19 17 16 ... 16  5  0]]


In [7]:
## Define a multi-layer LSTM network
rnn_size   = 512
num_layers = 3
grad_clip  = 5. # <= GRADIENT CLIPPING (PRACTICALLY IMPORTANT)
vocab_size = data_loader.vocab_size

# SELECT RNN CELL (MULTI LAYER LSTM)
unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)

# Set paths to the graph
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
initial_state = cell.zero_state(batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
#         inputs = tf.split(1, seq_length, tf.nn.embedding_lookup(embedding, input_data))    # Does not work in 1.x
        inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), seq_length, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
print ("Network ready")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Network ready


## Define functions

# Output of RNN
# outputs, last_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state
#                         , cell, loop_function=None, scope='rnnlm')
outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state
                        , cell, loop_function=None, scope='rnnlm')
# output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
output = tf.reshape(outputs, [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

# Next word probability
probs = tf.nn.softmax(logits)
print ("FUNCTIONS READY")

In [9]:
## Define loss function
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight
    vocab_size)
print ("LOSS FUNCTION")

LOSS FUNCTION


In [12]:
## Define cost function
cost = tf.reduce_sum(loss) / batch_size / seq_length

# GRADIENT CLIPPING ! 
lr = tf.Variable(0.0, trainable=False) # <= LEARNING RATE 
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

final_state = last_state
print ("NETWORK READY")

NETWORK READY


In [11]:
## Optimize with lr scheduling
## It takes LOOOOOOOOOOOONG time to train...
num_epochs    = 500
save_every    = 1000
learning_rate = 0.0002
decay_rate    = 0.97

save_dir = 'data/nine_dreams'
sess = tf.Session()
sess.run(tf.initialize_all_variables())    # Use tf.global_variables_initializer instead
summary_writer = tf.summary.FileWriter(save_dir
                    , graph=sess.graph)     # tf.train.SummaryWriter deprecated
saver = tf.train.Saver(tf.all_variables())
for e in range(num_epochs): # for all epochs

    # LEARNING RATE SCHEDULING 
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))

    data_loader.reset_batch_pointer()
    state = sess.run(initial_state)
    for b in range(data_loader.num_batches):
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {input_data: x, targets: y, initial_state: state}
        # Train!
        train_loss, state, _ = sess.run([cost, final_state, optm], feed)
        end = time.time()
        # PRINT 
        if b % 100 == 0:
            print ("%d/%d (epoch: %d), loss: %.3f, time/batch: %.3f"  
                   % (e * data_loader.num_batches + b
                    , num_epochs * data_loader.num_batches
                    , e, train_loss, end - start))
        # SAVE MODEL
        if (e * data_loader.num_batches + b) % save_every == 0:
            checkpoint_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path
                       , global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
0/99500 (epoch: 0), loss: 4.332, time/batch: 0.560
model saved to data/nine_dreams/model.ckpt
100/99500 (epoch: 0), loss: 2.952, time/batch: 0.052
199/99500 (epoch: 1), loss: 3.177, time/batch: 0.054
299/99500 (epoch: 1), loss: 2.951, time/batch: 0.053
398/99500 (epoch: 2), loss: 3.138, time/batch: 0.053
498/99500 (epoch: 2), loss: 2.951, time/batch: 0.055
597/99500 (epoch: 3), loss: 3.108, time/batch: 0.052
697/99500 (epoch: 3), loss: 2.951, time/batch: 0.057
796/99500 (epoch: 4), loss: 3.082, time/batch: 0.054
896/99500 (epoch: 4), loss: 2.951, time/batch: 0.058
995/99500 (epoch: 5), loss: 3.060, time/batch: 0.049
model saved to data/nine_dreams/model.ckpt
1095/99500 (epoch: 5), loss: 2.951, time/batch: 0.062
1194/99500 (epoch: 6), loss: 3.043, time/batch: 0.055
1294/99500 (epoch: 6), loss: 2.950, time/batch: 0.055
1393/99500 (epoch: 7), loss: 3

12836/99500 (epoch: 64), loss: 2.947, time/batch: 0.047
12935/99500 (epoch: 65), loss: 2.892, time/batch: 0.053
model saved to data/nine_dreams/model.ckpt
13035/99500 (epoch: 65), loss: 2.947, time/batch: 0.046
13134/99500 (epoch: 66), loss: 2.891, time/batch: 0.054
13234/99500 (epoch: 66), loss: 2.947, time/batch: 0.046
13333/99500 (epoch: 67), loss: 2.890, time/batch: 0.046
13433/99500 (epoch: 67), loss: 2.947, time/batch: 0.048
13532/99500 (epoch: 68), loss: 2.888, time/batch: 0.047
13632/99500 (epoch: 68), loss: 2.947, time/batch: 0.047
13731/99500 (epoch: 69), loss: 2.892, time/batch: 0.051
13831/99500 (epoch: 69), loss: 2.947, time/batch: 0.051
13930/99500 (epoch: 70), loss: 2.888, time/batch: 0.053
model saved to data/nine_dreams/model.ckpt
14030/99500 (epoch: 70), loss: 2.947, time/batch: 0.052
14129/99500 (epoch: 71), loss: 2.887, time/batch: 0.056
14229/99500 (epoch: 71), loss: 2.947, time/batch: 0.054
14328/99500 (epoch: 72), loss: 2.886, time/batch: 0.053
14428/99500 (epoch

26268/99500 (epoch: 132), loss: 2.869, time/batch: 0.051
26368/99500 (epoch: 132), loss: 2.947, time/batch: 0.050
26467/99500 (epoch: 133), loss: 2.869, time/batch: 0.047
26567/99500 (epoch: 133), loss: 2.947, time/batch: 0.053
26666/99500 (epoch: 134), loss: 2.869, time/batch: 0.048
26766/99500 (epoch: 134), loss: 2.947, time/batch: 0.052
26865/99500 (epoch: 135), loss: 2.869, time/batch: 0.051
26965/99500 (epoch: 135), loss: 2.947, time/batch: 0.044
model saved to data/nine_dreams/model.ckpt
27064/99500 (epoch: 136), loss: 2.869, time/batch: 0.053
27164/99500 (epoch: 136), loss: 2.947, time/batch: 0.054
27263/99500 (epoch: 137), loss: 2.868, time/batch: 0.053
27363/99500 (epoch: 137), loss: 2.947, time/batch: 0.048
27462/99500 (epoch: 138), loss: 2.868, time/batch: 0.045
27562/99500 (epoch: 138), loss: 2.947, time/batch: 0.055
27661/99500 (epoch: 139), loss: 2.868, time/batch: 0.047
27761/99500 (epoch: 139), loss: 2.947, time/batch: 0.055
27860/99500 (epoch: 140), loss: 2.868, time/b

39601/99500 (epoch: 199), loss: 2.865, time/batch: 0.041
39701/99500 (epoch: 199), loss: 2.947, time/batch: 0.044
39800/99500 (epoch: 200), loss: 2.865, time/batch: 0.041
39900/99500 (epoch: 200), loss: 2.947, time/batch: 0.044
39999/99500 (epoch: 201), loss: 2.865, time/batch: 0.046
model saved to data/nine_dreams/model.ckpt
40099/99500 (epoch: 201), loss: 2.947, time/batch: 0.043
40198/99500 (epoch: 202), loss: 2.865, time/batch: 0.050
40298/99500 (epoch: 202), loss: 2.947, time/batch: 0.045
40397/99500 (epoch: 203), loss: 2.865, time/batch: 0.040
40497/99500 (epoch: 203), loss: 2.947, time/batch: 0.040
40596/99500 (epoch: 204), loss: 2.865, time/batch: 0.045
40696/99500 (epoch: 204), loss: 2.947, time/batch: 0.044
40795/99500 (epoch: 205), loss: 2.865, time/batch: 0.045
40895/99500 (epoch: 205), loss: 2.947, time/batch: 0.043
40994/99500 (epoch: 206), loss: 2.865, time/batch: 0.044
model saved to data/nine_dreams/model.ckpt
41094/99500 (epoch: 206), loss: 2.947, time/batch: 0.042
41

52934/99500 (epoch: 266), loss: 2.865, time/batch: 0.048
model saved to data/nine_dreams/model.ckpt
53034/99500 (epoch: 266), loss: 2.947, time/batch: 0.044
53133/99500 (epoch: 267), loss: 2.865, time/batch: 0.049
53233/99500 (epoch: 267), loss: 2.947, time/batch: 0.043
53332/99500 (epoch: 268), loss: 2.865, time/batch: 0.059
53432/99500 (epoch: 268), loss: 2.947, time/batch: 0.042
53531/99500 (epoch: 269), loss: 2.865, time/batch: 0.055
53631/99500 (epoch: 269), loss: 2.947, time/batch: 0.045
53730/99500 (epoch: 270), loss: 2.865, time/batch: 0.043
53830/99500 (epoch: 270), loss: 2.947, time/batch: 0.044
53929/99500 (epoch: 271), loss: 2.865, time/batch: 0.040
model saved to data/nine_dreams/model.ckpt
54029/99500 (epoch: 271), loss: 2.947, time/batch: 0.046
54128/99500 (epoch: 272), loss: 2.865, time/batch: 0.044
54228/99500 (epoch: 272), loss: 2.947, time/batch: 0.042
54327/99500 (epoch: 273), loss: 2.865, time/batch: 0.057
54427/99500 (epoch: 273), loss: 2.947, time/batch: 0.043
54

66267/99500 (epoch: 333), loss: 2.865, time/batch: 0.049
66367/99500 (epoch: 333), loss: 2.947, time/batch: 0.042
66466/99500 (epoch: 334), loss: 2.865, time/batch: 0.044
66566/99500 (epoch: 334), loss: 2.947, time/batch: 0.048
66665/99500 (epoch: 335), loss: 2.865, time/batch: 0.044
66765/99500 (epoch: 335), loss: 2.947, time/batch: 0.049
66864/99500 (epoch: 336), loss: 2.865, time/batch: 0.046
66964/99500 (epoch: 336), loss: 2.947, time/batch: 0.047
model saved to data/nine_dreams/model.ckpt
67063/99500 (epoch: 337), loss: 2.865, time/batch: 0.042
67163/99500 (epoch: 337), loss: 2.947, time/batch: 0.049
67262/99500 (epoch: 338), loss: 2.865, time/batch: 0.041
67362/99500 (epoch: 338), loss: 2.947, time/batch: 0.043
67461/99500 (epoch: 339), loss: 2.865, time/batch: 0.052
67561/99500 (epoch: 339), loss: 2.947, time/batch: 0.049
67660/99500 (epoch: 340), loss: 2.865, time/batch: 0.047
67760/99500 (epoch: 340), loss: 2.947, time/batch: 0.050
67859/99500 (epoch: 341), loss: 2.865, time/b

79600/99500 (epoch: 400), loss: 2.865, time/batch: 0.043
79700/99500 (epoch: 400), loss: 2.947, time/batch: 0.047
79799/99500 (epoch: 401), loss: 2.865, time/batch: 0.043
79899/99500 (epoch: 401), loss: 2.947, time/batch: 0.048
79998/99500 (epoch: 402), loss: 2.865, time/batch: 0.047
model saved to data/nine_dreams/model.ckpt
80098/99500 (epoch: 402), loss: 2.947, time/batch: 0.043
80197/99500 (epoch: 403), loss: 2.865, time/batch: 0.041
80297/99500 (epoch: 403), loss: 2.947, time/batch: 0.045
80396/99500 (epoch: 404), loss: 2.865, time/batch: 0.046
80496/99500 (epoch: 404), loss: 2.947, time/batch: 0.042
80595/99500 (epoch: 405), loss: 2.865, time/batch: 0.040
80695/99500 (epoch: 405), loss: 2.947, time/batch: 0.043
80794/99500 (epoch: 406), loss: 2.865, time/batch: 0.039
80894/99500 (epoch: 406), loss: 2.947, time/batch: 0.044
80993/99500 (epoch: 407), loss: 2.865, time/batch: 0.047
model saved to data/nine_dreams/model.ckpt
81093/99500 (epoch: 407), loss: 2.947, time/batch: 0.039
81

92933/99500 (epoch: 467), loss: 2.865, time/batch: 0.048
model saved to data/nine_dreams/model.ckpt
93033/99500 (epoch: 467), loss: 2.947, time/batch: 0.042
93132/99500 (epoch: 468), loss: 2.865, time/batch: 0.046
93232/99500 (epoch: 468), loss: 2.947, time/batch: 0.043
93331/99500 (epoch: 469), loss: 2.865, time/batch: 0.040
93431/99500 (epoch: 469), loss: 2.947, time/batch: 0.050
93530/99500 (epoch: 470), loss: 2.865, time/batch: 0.047
93630/99500 (epoch: 470), loss: 2.947, time/batch: 0.049
93729/99500 (epoch: 471), loss: 2.865, time/batch: 0.042
93829/99500 (epoch: 471), loss: 2.947, time/batch: 0.045
93928/99500 (epoch: 472), loss: 2.865, time/batch: 0.043
model saved to data/nine_dreams/model.ckpt
94028/99500 (epoch: 472), loss: 2.947, time/batch: 0.046
94127/99500 (epoch: 473), loss: 2.865, time/batch: 0.045
94227/99500 (epoch: 473), loss: 2.947, time/batch: 0.045
94326/99500 (epoch: 474), loss: 2.865, time/batch: 0.042
94426/99500 (epoch: 474), loss: 2.947, time/batch: 0.046
94